# Interactive Research Map Visualization

This notebook creates an interactive research map using DataMapPlot:
1. **Data Integration**: Combine UMAP coordinates with LLM-generated summaries
2. **Interactive Visualization**: Create multi-layer clustered research map
3. **Custom Tooltips**: Rich hover information with paper details
4. **Touch Optimization**: Modify interactions for touch-friendly navigation

**Input**: Clustered papers with coordinates and summaries  
**Output**: Interactive HTML research map for exploration

## 1. Setup and Data Loading

In [ ]:
import pandas as pd
import json
import numpy as np
import matplotlib
import datamapplot

# Configure matplotlib for better rendering
matplotlib.rcParams["figure.dpi"] = 72

# Load datasets
print("📂 Loading datasets...")
umap_df = pd.read_csv("umap_cluster_2d.csv")
with open("final_paper_summaries.json", 'r') as f:
    data_df = pd.DataFrame(json.load(f))

print(f"✅ UMAP data: {umap_df.shape}")
print(f"✅ Summary data: {data_df.shape}")

# Prepare visualization data
coordinates = umap_df[["umapX", "umapY"]].fillna(0).to_numpy()
layer_1 = umap_df["5L_Layer_1"].fillna("").to_numpy()
layer_2 = umap_df["5L_Layer_2"].fillna("").to_numpy()
layer_3 = umap_df["5L_Layer_3"].fillna("").to_numpy()
layer_4 = umap_df["5L_Layer_4"].fillna("").to_numpy()
layer_5 = umap_df["5L_Layer_5_Final"].fillna("").to_numpy()

print(f"🗺️  Map coordinates shape: {coordinates.shape}")

## 2. Prepare Rich Tooltip Data

Format the paper metadata for rich hover tooltips with visual tags and structured information.

In [ ]:
def format_tags_as_html(tags_list, color):
    """Convert list of tags to styled HTML elements"""
    if not isinstance(tags_list, list) or not tags_list:
        return ""

    tag_html_elements = []
    for tag in tags_list:
        tag_html = f'''
        <div style="
            background-color: {color};
            color: #fff;
            border-radius: 6px;
            padding: 2px 10px;
            font-size: 7pt;
            font-weight: 600;
            display: inline-block;
        ">{tag}</div>
        '''
        tag_html_elements.append(tag_html)

    return f'<div style="display: flex; flex-wrap: wrap; gap: 5px;">{"".join(tag_html_elements)}</div>'

# Format technique and concept tags
data_df["techniques_html"] = data_df["techniques_tools"].apply(
    lambda x: format_tags_as_html(x, "#2a5982")
)
data_df["concepts_html"] = data_df["key_concepts"].apply(
    lambda x: format_tags_as_html(x, "#d6ac4b")
)

print("✅ Formatted tooltip tags")

## 3. Custom Tooltip Design

Create a professional, information-rich tooltip template for paper visualization.

In [ ]:
# Custom tooltip CSS for clean appearance
tooltip_css = """
    max-width: none !important;
    width: auto !important;
    background: transparent !important;
    box-shadow: none !important;
    padding: 0 !important;
    border: none !important;
    pointer-events: none;
"""

# Rich tooltip template with two-column layout
tooltip_template = """
<div style="
    font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Helvetica, Arial, sans-serif;
    width: 750px;
    background: #fff;
    border: 1px solid #e5e7eb;
    border-radius: 10px;
    box-shadow: 0 8px 24px rgba(0,0,0,0.12);
    overflow: hidden;
    color: #0f172a;
    text-align: left;
    position: relative;
">
    <!-- Left Column: Abstract & Metadata -->
    <div style="
        position: absolute;
        top: 0; left: 0; bottom: 0;
        width: 58%;
        padding: 20px;
        overflow-y: auto;
        box-sizing: border-box;
    ">
        <h2 style="margin: 0 0 6px 0; font-size: 16px; font-weight: 700; color: #1e293b; line-height: 1.3;">
            {hover_text}
        </h2>
        
        <div style="font-size: 11px; color: #64748b; font-weight: 600; margin-bottom: 6px;">
            Published: {year}
        </div>
        
        <div style="display: flex; flex-wrap: wrap; gap: 4px; margin-bottom: 6px;">
            <div style="background-color: #2a5982; color: #fff; border-radius: 6px; padding: 2px 10px; font-size: 7pt; font-weight: 600;">
                {domain}
            </div>
            <div style="background-color: #d6ac4b; color: #fff; border-radius: 6px; padding: 2px 10px; font-size: 7pt; font-weight: 600;">
                {data_type}
            </div>
        </div>

        <div style="font-size: 11px; font-weight: 700; color: #2a5982; text-transform: uppercase; margin-bottom: 6px; letter-spacing: 0.5px;">
            Abstract
        </div>
        <p style="margin: 0; font-size: 12.5px; line-height: 1.5; color: #334155; text-align: justify;">
            {original_abstract}
        </p>
    </div>

    <!-- Right Column: Summaries & Analysis -->
    <div style="
        margin-left: 58%;
        width: 42%;
        padding: 20px;
        background-color: #f8fafc;
        border-left: 1px solid #e2e8f0;
        box-sizing: border-box;
        height: fit-content;
    ">
        <div style="margin-bottom: 16px;">
            <div style="font-size: 12px; font-weight: 700; color: #2a5982; margin-bottom: 6px;">Simple Explanation</div>
            <p style="margin: 0; font-size: 12.5px; line-height: 1.45; color: #0f172a;">{summary_simple}</p>
        </div>

        <div style="margin-bottom: 16px;">
            <div style="font-size: 12px; font-weight: 700; color: #2a5982; margin-bottom: 6px;">Key Findings</div>
            <p style="margin: 0; font-size: 12.5px; line-height: 1.45; color: #0f172a;">{main_findings}</p>
        </div>

        <div style="margin-bottom: 16px;">
            <div style="font-size: 12px; font-weight: 700; color: #2a5982; margin-bottom: 6px;">Methodology</div>
            <p style="margin: 0; font-size: 12.5px; line-height: 1.45; color: #0f172a;">{methodology}</p>
        </div>

        <div style="margin-bottom: 16px;">
            <div style="font-size: 12px; font-weight: 700; color: #2a5982; margin-bottom: 5px;">Techniques & Tools</div>
            {techniques_html}
        </div>

        <div>
            <div style="font-size: 12px; font-weight: 700; color: #d6ac4b; margin-bottom: 5px;">Key Concepts</div>
            {concepts_html}
        </div>
    </div>
</div>
"""

print("✅ Tooltip template configured")

## 4. Create Interactive Research Map

In [ ]:
# Prepare data columns for tooltip
tooltip_columns = [
    "title", "year", "domain", "summary_simple", "main_findings",
    "original_abstract", "methodology", "data_type", "techniques_html", "concepts_html"
]

# Create the interactive plot
print("🎨 Creating interactive research map...")
interactive_plot = datamapplot.create_interactive_plot(
    # Coordinates and labels
    coordinates,
    layer_1, layer_2, layer_3, layer_4, layer_5,
    
    # Content and appearance
    hover_text=data_df["title"].fillna(""),
    noise_label="",
    cluster_boundary_line_width=6,
    color_label_text=False,
    
    # Metadata
    title="MUREX Research Map",
    sub_title="Mahidol University Research Papers (2021-2025)",
    logo="https://mahidol.ac.th/documents/CI/update/Mahidol/10.png",
    logo_width=128,
    
    # Interactive features
    enable_search=True,
    extra_point_data=data_df[tooltip_columns].fillna(""),
    hover_text_html_template=tooltip_template,
    tooltip_css=tooltip_css,
    
    # Timeline histogram
    histogram_data=pd.to_datetime("06/" + data_df['year'].astype(str), format='%m/%Y'),
    histogram_group_datetime_by="year",
    histogram_range=(pd.to_datetime("2021-01-01"), pd.to_datetime("2026-01-01"))
)

print("✅ Interactive plot created successfully")
interactive_plot

## 5. Export and Touch Optimization

In [ ]:
# Save the interactive map
interactive_plot.save("murex_research_map.html")
print("💾 Saved as murex_research_map.html")

def optimize_for_touch_devices(filename):
    """Modify the HTML to be more touch-friendly"""
    
    print(f"🔧 Optimizing {filename} for touch devices...")
    
    try:
        with open(filename, "r", encoding="utf-8") as f:
            content = f.read()
    except FileNotFoundError:
        print(f"❌ File '{filename}' not found")
        return

    # Touch-friendly interaction modifications
    modifications = [
        {
            "name": "Disable original click events",
            "search": ".on('click',e=>this.#handleClick(e))",
            "replace": ".on('disabled_click',e=>this.#handleClick(e))"
        },
        {
            "name": "Convert mousemove to click for touch",
            "search": ".on('mousemove',e=>this.#handleMouseMove(e))",
            "replace": ".on('click',e=>this.#handleMouseMove(e))"
        },
        {
            "name": "Convert mouseleave to doubleclick for touch",
            "search": ".on('mouseleave',e=>this.#handleMouseLeave(e))",
            "replace": ".on('dblclick',e=>this.#handleMouseLeave(e))"
        }
    ]

    changes_applied = 0

    for mod in modifications:
        if mod["search"] in content:
            content = content.replace(mod["search"], mod["replace"])
            changes_applied += 1
            print(f"   ✅ {mod['name']}")
        else:
            print(f"   ⚠️  Could not find: {mod['name']}")

    if changes_applied > 0:
        with open(filename, "w", encoding="utf-8") as f:
            f.write(content)
        print(f"🎯 Applied {changes_applied} touch optimizations")
    else:
        print("⚠️  No modifications applied")

# Apply touch optimizations
optimize_for_touch_devices("murex_research_map.html")

print("\n🎉 Interactive research map creation complete!")
print("📱 Optimized for both desktop and mobile devices")
print("🔍 Features:")
print("   - Multi-layer cluster navigation")
print("   - Rich paper tooltips")
print("   - Year-based timeline filtering")
print("   - Full-text search capabilities")
print("   - Touch-friendly interactions")